In [1]:
# Installing dependencies
#!pip install praw
#!pip install psaw
#!pip install yfinance

In [2]:
import praw
from psaw import PushshiftAPI
import json
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import traceback
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import os

# Suppress warning messages
import warnings
#warnings.filterwarnings('ignore')

In [3]:
# Load client_id, secret_id, and user_agent
with open('info.json') as f:
     info = json.load(f)
        
info = dict(info)

In [4]:
# Initialize Reddit and PushshiftAPI instances
reddit = praw.Reddit(client_id=info["client_id"], user_agent=info["user_agent"], client_secret=info["client_secret"])
api = PushshiftAPI(reddit)

In [5]:
# Store results of a search in a DataFrame
"""
subm_dicts = [{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in api.search_submissions(subreddit='stocks', q="TWTR", filter=['url','author', 'title', 'subreddit'], limit=100)]
df = pd.DataFrame(subm_dicts)
df
"""
_ = None

In [6]:
# Get historical stock data for a ticker
#twtr = yf.download('TWTR', progress=True)
#twtr

## Processing Tickers

This initial thought process is not great. I decided that looking for specific words wwould not be a good idea, as it takes much of the context out of the comment. (Something that briefly mentions TSLA, but is actually talking about how great MSFT is would be useless in predicting TSLA stock)

In [7]:
# At close, calculate the real and percent change since last close
def get_diff(ticker_data):
    df = ticker_data.copy()
    real = []
    percent = []
    for index, row in df.reset_index().iterrows():
        if(index == 0):
            real.append(0)
            percent.append(0)
        else:
            real.append(row["Close"]-df.iloc[index-1]["Close"])
            percent.append(real[-1]/df.iloc[index-1]["Close"])
    return real, percent

# Get the reddit posts that mention a certain ticker n days before a large change in stock price
def get_pre_change_posts(ticker, ticker_gain, days=1, limit=1000, subreddit="stocks,stockmarket,stocksandtrading,daytrading,investing,stocks_picks,stockstobuytoday"):
    df = None
    for index, row in ticker_gain.iterrows():
        start_date = datetime.fromtimestamp(row["Date"].timestamp()) + timedelta(hours=6, days=-days)
        end_date = datetime.fromtimestamp(row["Date"].timestamp()) + timedelta(hours=6)
        
        # TODO: Check whether comments would be better than submissions
        
        submissions = api.search_comments(after=start_date, before=end_date, q=ticker, subreddit=subreddit, filter=['url','author', 'title', 'subreddit'], limit=limit)
        if(df is None):
            df = pd.DataFrame([{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in submissions])
        else:
            df = df.append([{k:getattr(praw_obj, k) for k in vars(praw_obj)} for praw_obj in submissions], ignore_index=True)
    return df

def word_counts(df, column="body", min_letters=3):
    counts = {}
    for i in list(df[column]):
        for j in i.split(" "):
            j = ''.join(k for k in j if k.isalnum())
            # Exclude words that are likely tickers
            if(j == j.upper() and len(j) > 1 and len(j) <= 5):
                pass
            elif(len(j) < 3):
                pass
            elif(j not in counts.keys()):
                counts[j.lower()] = 1
            else:
                counts.update({j.lower():counts.get(j.lower())+1})
    return counts

def remove_shared_keys(dict_a, dict_b, cutoff=2):
    a = dict_a.copy()
    b = dict_b.copy()
    
    rm_a = []
    rm_b = []
    for i in a:
        if(i in b):
            if(b.get(i) > 2*a.get(i)):
                rm_a.append(i)
            elif(b.get(i) < 2*a.get(i)):
                rm_b.append(i)
            else:
                rm_a.append(i)
                rm_b.append(i)
    for i in rm_a:
        a.pop(i)
    for i in rm_b:
        b.pop(i)
    return a, b

def remove_infrequent_words(dict_a, min_count=2):
    d = dict_a.copy()
    
    to_remove = []
    for i, x in d.items():
        if(x < min_count):
            to_remove.append(i)
            
    for i in to_remove:
        d.pop(i)
        
    return d

# Generate information for a given ticker
def process_ticker(ticker, gain_cutoff=0.05, loss_cutoff=0.05, limit=100, days=1):
    try:
        ticker_data = yf.download(ticker, progress=False)
        ticker_data.reset_index(inplace=True)
        real, percent = get_diff(ticker_data)

        ticker_data["Real_Change"] = real
        ticker_data["Percent_Change"] = percent

        ticker_gain = ticker_data[ticker_data["Percent_Change"] > gain_cutoff]
        ticker_loss = ticker_data[ticker_data["Percent_Change"] < -loss_cutoff]

        pre_gain = get_pre_change_posts(ticker, ticker_gain, days, limit)
        pre_loss = get_pre_change_posts(ticker, ticker_loss, days, limit)

        gain_wc = dict(sorted(word_counts(pre_gain).items(), key=lambda x: x[1], reverse=True))
        loss_wc = dict(sorted(word_counts(pre_loss).items(), key=lambda x: x[1], reverse=True))

        gain_freq = remove_infrequent_words(gain_wc)
        loss_freq = remove_infrequent_words(loss_wc)

        gain_only, loss_only = remove_shared_keys(gain_freq, loss_freq)
    except Exception as e:
        print(e)
        return {}, {}
    
    return gain_only, loss_only

#gain_only, loss_only = process_ticker("FB")

#gain_only

#loss_only
"""
all_gain = []
all_loss = []
for ticker in ["TWTR", "FB", "MSFT", "ADBE", "AAPL", "SNAP", "AMZN", "NCL", "DIS", "NFLX"]:
    gain_only, loss_only = process_ticker(ticker)
    all_gain.append(gain_only)
    all_loss.append(loss_only)
    print(ticker)

def combine_dict_list(list_of_dicts):
    single_dict = {}
    for d in list_of_dicts:
        for i in d:
            if(i not in single_dict):
                single_dict[i] = d.get(i)
            else:
                single_dict.update({i:single_dict.get(i)+d.get(i)})
    return dict(sorted(single_dict.items(), key=lambda x: x[1], reverse=True))

gain, loss = remove_shared_keys(combine_dict_list(all_gain), combine_dict_list(all_loss))

gain

loss"""
_ = None # This is just to stop automatic output of block commented code

## Better Method (Probably)

Instead of looking at posts/comments the day before and predicting whether the next day will close higher, this will be looking at the posts/comments from the previous day's close to the current day's open and predicting whether the close price will be higher than the open price. 

In [8]:
def daily_change(data):
    change = []
    up = []
    for index, row in data.iterrows():
        change.append(row["Close"]-row["Open"])
        up.append(int(change[-1] > 0))
    return change, up

In [9]:
#change, up = daily_change(twtr)

In [10]:
#twtr["Daily_Change"] = change
#twtr["Positive_Change"] = up

In [11]:
#twtr = twtr.sample(frac=1)

In [12]:
#twtr

In [13]:
def get_pre_open_content(data, ticker, start_hour_diff=0, subreddit="stocks,stockmarket,stocksandtrading,daytrading,investing,stocks_picks,stockstobuytoday", limit=100):
    new_col = []
    for index, row in data.iterrows():
        end_time = row.name + timedelta(hours=9, minutes=30)
        start_time = end_time - timedelta(hours=17, minutes=30)
        content = []
        for i in api.search_comments(after=start_time, before=end_time, subreddit=subreddit, q=ticker, filter=['url','author', 'title', 'subreddit'], limit=limit):
            for j in i.body.split("."):
                for k in j.split("\n"):
                    content.append(k)
        new_col.append(content)
    
    return new_col

In [14]:
# This takes quite a while to run. Looking into how to speed it up. 
#twtr_full = twtr.copy()
#twtr = twtr.head(10)
#twtr["Comments"] = get_pre_open_content(twtr, "TWTR", limit=100)

In [15]:
#twtr

### Preparing text for analysis

In [16]:
def split_sequences(data):
    dates = []
    seqs = []
    vals = []
    
    for index, row in data.iterrows():
        for comment in row["Comment_Sequences"]:
            if(comment != []):
                dates.append(row.name)
                seqs.append(list(comment))
                vals.append(row["Positive_Change"])
    return pd.DataFrame({"Date":dates, "Sequence":seqs, "Positive_Change":vals})

In [55]:
def prepare_text(data, train_proportion = 0.8, max_len=50, tokenizer=None):
    if(tokenizer is None):
        tokenizer = Tokenizer(oov_token = "<OOV>")
    
    train = data[:int(data.shape[0]*train_proportion)]
    test = data[int(data.shape[0]*train_proportion):]
    
    for comment in train.Comments:
        tokenizer.fit_on_texts(comment)
        
    seqs = []
    for comment in train.Comments:
        seqs.append(tokenizer.texts_to_sequences(comment))
    
    train["Comment_Sequences"] = seqs
    
    train = split_sequences(train)
        
    train_padded = pad_sequences(train["Sequence"], padding="post", truncating="post", maxlen=max_len)

    for comment in test.Comments:
        tokenizer.fit_on_texts(comment)
        
    seqs = []
    for comment in test.Comments:
        seqs.append(tokenizer.texts_to_sequences(comment))
    
    test["Comment_Sequences"] = seqs
    
    test = split_sequences(test)
        
    test_padded = pad_sequences(test["Sequence"], padding="post", truncating="post", maxlen=max_len)
    
    return train, train_padded, test, test_padded, tokenizer

In [56]:
def generate_ticker_data(tickers = None):
    if(tickers is None): return None
    
    cur_tokenizer = None
    
    for ticker in tickers:
        if(f"{ticker}.pkl" in os.listdir('data/')):
            continue
        
        data = yf.download(ticker)
        change, up = daily_change(data)
        data["Daily_Change"] = change
        data["Positive_Change"] = up
        data = data.sample(frac=1)
        
        data["Comments"] = get_pre_open_content(data, ticker, limit=100)
        
        train, train_padded, test, test_padded, cur_tokenizer = prepare_text(data, tokenizer=cur_tokenizer)
        
        with open(f"data/{ticker}.pkl", "wb+") as f:
            pickle.dump((train, train_padded, test, test_padded), f)
            
        used_tickers = "_".join(tickers[:tickers.index(ticker)+1])
        
        with open(f"data/tokenizer/tokenizer_{used_tickers}.pkl", "wb+") as f:
            pickle.dump(tokenizer, f)

In [63]:
#generate_ticker_data(["TWTR"])

In [64]:
#with open("data/TWTR.pkl", "rb") as f:
    #train, train_padded, test, test_padded = pickle.load(f)

In [65]:
#type(train_padded[0][0])

## Loading all ticker data from pickle files

In [98]:
with open("data/tokenizer/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

all_data = []
for file in os.listdir("data"):
    if(file != "tokenizer"):
        with open(f"data/{file}", "rb") as f:
            all_data.append(pickle.load(f))

In [99]:
# Combine all sequences
X_train = None
X_test = None
for i in all_data:
    if(X_train is None):
        X_train = i[1]
        X_test = i[3]
    else:
        X_train = np.vstack((X_train, i[1]))
        X_test = np.vstack((X_test, i[3]))
X_train = np.array(X_train)
X_test = np.array(X_test)

In [100]:
X_train.shape

(915501, 50)

In [101]:
X_test.shape

(229221, 50)

In [102]:
unioned_train_data = None
unioned_test_data = None
for i in all_data:
    if(unioned_train_data is None):
        unioned_train_data = i[0]
        unioned_test_data = i[2]
    else:
        unioned_train_data = pd.concat([unioned_train_data, i[0]], ignore_index=True)
        unioned_test_data = pd.concat([unioned_test_data, i[2]], ignore_index=True)
y_train = unioned_train_data["Positive_Change"]
y_test = unioned_test_data["Positive_Change"]

In [103]:
len(y_train)

915501

In [104]:
len(y_test)

229221

### Creating TensorFlow Network

In [105]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(len(tokenizer.word_index)+1, 16, input_length=len(X_train[0])))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(24, activation='relu'))
model.add(tf.keras.layers.Dense(48, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [106]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 16)            1403056   
                                                                 
 global_average_pooling1d_4   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 24)                408       
                                                                 
 dense_13 (Dense)            (None, 48)                1200      
                                                                 
 dense_14 (Dense)            (None, 1)                 49        
                                                                 
Total params: 1,404,713
Trainable params: 1,404,713
Non-trainable params: 0
____________________________________________

In [107]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  229/28610 [..............................] - ETA: 5:47 - loss: 0.6928 - accuracy: 0.5123

KeyboardInterrupt: 

In [88]:
pred = model.predict(X_test)

In [89]:
real_pred = []
for i in pred:
    real_pred.append(1 if i > 0.5 else 0)

In [90]:
from sklearn.metrics import accuracy_score

In [91]:
accuracy_score(real_pred, y_test)

0.4926895848720839

### Evaluating Model

In [92]:
def evaluate(test_data, padded, model):
    days = []
    correct_proportion = []
    pred_sum = []
    
    for date in test_data.Date.unique():
        days.append(date)
        
        day = test_data[test_data["Date"] == date]
                
        day_padded = padded[day.index[0]:day.index[-1]+1]
        
        pred = model.predict(day_padded)
        
        real_pred = []
        for i in pred:
            real_pred.append(1 if i > 0.5 else 0)
            
        exp = list(day["Positive_Change"])[0]
        pred_sum.append(sum(real_pred))
        correct_count = 0
        for i in real_pred:
            if(i == exp):
                correct_count += 1
        correct_proportion.append(correct_count/len(real_pred))
    return pd.DataFrame({"Date":days, "Correct_Proportion":correct_proportion, "Prediction_Sum":pred_sum})

In [97]:
results = evaluate(unioned_test_data, X_test, model)
results

KeyboardInterrupt: 

In [ ]:
results.Correct_Proportion.hist()